In [23]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("HUGGINGFACE_API_KEY")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key,
    temperature=0.1,
    max_new_tokens=100
 )

model = ChatHuggingFace(llm=llm)

class Person(BaseModel):
    name: str = Field(description="The person's name")
    age: int = Field(description="The person's age")
    occupation: str = Field(description="The person's occupation") 

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template="""You are a JSON generator. Generate ONLY valid JSON for a {place} person.

{formate_intructions}

IMPORTANT RULES:
- Return ONLY the JSON object
- NO code, NO explanations, NO markdown
- Start directly with {{ and end with }}

Example output format:
{{"name": "Ahmed", "age": 35, "occupation": "Engineer"}}

Now generate for {place}:""",
    input_variables=["place"],
    partial_variables={'formate_intructions': parser.get_format_instructions()}
)



chain = (
    template 
    | model 
    | parser
)

response = chain.invoke({"place": "Pakistan"})
print("Response from model:")
print(response)


Response from model:
name='Pakistan' age=75 occupation='Country'
